In [ ]:
import numpy as np
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
import sys

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

from hidden_points_removal import hidden_point_removal_o3d, hidden_point_removal_biasutti
from point_cloud_utils import transform_pcd, filter_points_from_dict, get_pcd, point_to_label, change_point_indices
from point_to_pixels import point_to_pixel
from visualization_utils import unite_pcd_and_img, color_pcd_with_labels, visualize_associations_in_img
from merge_pointclouds import build_associations, apply_associations_to_dict, merge_label_predictions, merge_pointclouds, build_associations_across_timesteps
from merged_sequences import merged_sequence
from image_utils import masks_to_image, masks_to_colored_image

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(2.5, 120),
            ApplyPose(),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we read in the point cloud and the left and right image of the stereo camera. If labels for those images are available they can be read in, too!

In [ ]:
points_index = 100
left_cam = "cam2"
right_cam = "cam3"

pcd_o3d = get_pcd(dataset.get_point_cloud(points_index))
pcd = np.asarray(pcd_o3d.points)

left_image_PIL = dataset.get_image(left_cam, points_index)
left_image = cv2.cvtColor(np.array(left_image_PIL), cv2.COLOR_RGB2BGR)

right_image_PIL = dataset.get_image(right_cam, points_index)
right_image = cv2.cvtColor(np.array(right_image_PIL), cv2.COLOR_RGB2BGR)

# Only do this, if there is a SAM label present for respective point_index

left_label_PIL = dataset.get_sam_label(left_cam, points_index)
left_label = cv2.cvtColor(np.array(left_label_PIL), cv2.COLOR_RGB2BGR)

right_label_PIL = dataset.get_sam_label(right_cam, points_index)
right_label = cv2.cvtColor(np.array(right_label_PIL), cv2.COLOR_RGB2BGR)

left_masks = dataset.get_sam_mask(left_cam, points_index)
left_labels_from_mask = masks_to_image(left_masks)
colored_masks_left = masks_to_colored_image(left_masks).astype(int)

right_masks = dataset.get_sam_mask(right_cam, points_index)
right_labels_from_mask = masks_to_image(right_masks)
colored_masks_right = masks_to_colored_image(right_masks).astype(int)

#Otherwise use this:
#right_label = None
#left_label = None

Now we read in the transformation matrixes and intriniscs for the two cameras and transform the point cloud accordingly!

In [ ]:
T_lidar2leftcam, K_leftcam = dataset.get_calibration_matrices(left_cam)
T_lidar2rightcam, K_rightcam = dataset.get_calibration_matrices(right_cam)

pcd_leftcamframe = transform_pcd(pcd, T_lidar2leftcam)
pcd_rightcamframe = transform_pcd(pcd, T_lidar2rightcam)

Here we perform hidden point removal on the two camframes, since different points are hidden from different perspectives!

In [ ]:
hpr_mode = "o3d" # "o3d" or "biscutti"

if hpr_mode == "o3d":
    #Camera set to [0,0,0] because we are already in camera frame and camera position is origin
    hpr_mask_leftcam = hidden_point_removal_o3d(pcd_leftcamframe, camera=[0,0,0], radius_factor=400) 
    hpr_mask_rightcam = hidden_point_removal_o3d(pcd_rightcamframe, camera=[0,0,0], radius_factor=400) 
elif hpr_mode == "biasutti":
    hpr_mask_leftcam = hidden_point_removal_biasutti(pcd_leftcamframe, n_neighbours=64)
    hpr_mask_rightcam = hidden_point_removal_biasutti(pcd_rightcamframe, n_neighbours=64)

pcd_leftcamframe_hpr = pcd_leftcamframe[hpr_mask_leftcam]
pcd_rightcamframe_hpr = pcd_rightcamframe[hpr_mask_rightcam]

Here we build up point-to-pixel correspondences from the pointclouds to the respective image frames!

In [ ]:
point_to_pixel_dict_leftcam = point_to_pixel(pcd_leftcamframe, K_leftcam, left_image.shape[0], left_image.shape[1])
point_to_pixel_dict_rightcam = point_to_pixel(pcd_rightcamframe, K_rightcam, right_image.shape[0], right_image.shape[1])

point_to_pixel_dict_hpr_leftcam = point_to_pixel(pcd_leftcamframe_hpr, K_leftcam, left_image.shape[0], left_image.shape[1])
point_to_pixel_dict_hpr_rightcam = point_to_pixel(pcd_rightcamframe_hpr, K_rightcam, right_image.shape[0], right_image.shape[1])

In [ ]:
print("Number of points in initial point cloud: ", pcd.shape[0])
print("Number of points after Hidden Point Removal from perspective of left camera: ", pcd_leftcamframe_hpr.shape[0])
print("Number of points after Hidden Point Removal from perspective of right camera: ", pcd_rightcamframe_hpr.shape[0])
print("Number of points projected on left image without Hidden Point Removal: ", len(point_to_pixel_dict_leftcam))
print("Number of points projected on right image without Hidden Point Removal: ", len(point_to_pixel_dict_rightcam))
print("Number of points projected on left image with Hidden Point Removal: ", len(point_to_pixel_dict_hpr_leftcam))
print("Number of points projected on right image with Hidden Point Removal: ", len(point_to_pixel_dict_hpr_rightcam))

Here we unite the point clouds with the images and color the point clouds according to the available labels or the depth of the points onto the images!

In [ ]:
# Coloring either with depth or label_map, depending on wether label is available
left_img_overlay = unite_pcd_and_img(point_to_pixel_dict_leftcam, left_image, left_labels_from_mask, coloring="label_map", is_instance=True)
right_img_overlay = unite_pcd_and_img(point_to_pixel_dict_hpr_rightcam, right_image, right_labels_from_mask, coloring="label_map", is_instance=True)

left_img_overlay_hpr = unite_pcd_and_img(point_to_pixel_dict_hpr_leftcam, left_image, left_labels_from_mask, coloring="label_map", is_instance=True)
right_img_overlay_hpr = unite_pcd_and_img(point_to_pixel_dict_hpr_rightcam, right_image, right_labels_from_mask, coloring="label_map", is_instance=True)


In [ ]:
### The color values are random!!! ###

fig, axes = plt.subplots(4, 2)

images = [left_image, right_image, left_label, right_label, left_img_overlay, right_img_overlay, left_img_overlay_hpr, right_img_overlay_hpr]

titles = ["Left Image", "Right Image", "Left Label", "Right Label", "Left Image Overlay", "Right Image Overlay", "Left Image Overlay HPR", "Right Image Overlay HPR"]

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title(titles[i], fontsize=6)
    ax.set_xticks([]), ax.set_yticks([])

fig.set_dpi(300)

plt.show()

Here we filter the point clouds according to the points that are visible within the image frames!

In [ ]:
pcd_leftcamframe_fov = filter_points_from_dict(pcd, point_to_pixel_dict_leftcam)
pcd_rightcamframe_fov = filter_points_from_dict(pcd, point_to_pixel_dict_rightcam)

pcd_leftcamframe_fov_hpr = filter_points_from_dict(pcd_leftcamframe_hpr, point_to_pixel_dict_hpr_leftcam)
pcd_rightcamframe_fov_hpr = filter_points_from_dict(pcd_rightcamframe_hpr, point_to_pixel_dict_hpr_rightcam)

For each of those points, we build up correspondences that map the point index to label color!

In [ ]:
point_to_label_dict_leftcam = point_to_label(point_to_pixel_dict_leftcam, left_label)
point_to_label_dict_rightcam = point_to_label(point_to_pixel_dict_rightcam, right_label)

point_to_label_dict_hpr_leftcam = point_to_label(point_to_pixel_dict_hpr_leftcam, left_label)
point_to_label_dict_hpr_rightcam = point_to_label(point_to_pixel_dict_hpr_rightcam, right_label)

Using those correspondences, we can now color the point clouds with the colors from the labels!

In [ ]:
pcd_leftcamframe_fov = color_pcd_with_labels(pcd_leftcamframe_fov, point_to_label_dict_leftcam)
pcd_rightcamframe_fov = color_pcd_with_labels(pcd_rightcamframe_fov, point_to_label_dict_rightcam)

pcd_leftcamframe_fov_hpr = color_pcd_with_labels(pcd_leftcamframe_fov_hpr, point_to_label_dict_hpr_leftcam)
pcd_rightcamframe_fov_hpr = color_pcd_with_labels(pcd_rightcamframe_fov_hpr, point_to_label_dict_hpr_rightcam)

In [ ]:
#o3d.visualization.draw_geometries([pcd_rightcamframe_fov])

Now, we would like to join the instance label predictions from left and right camera.
As a first step, we build associations between the colors of the instance labels from left & right camera!

In [ ]:
# If we want to do that with the filtered point cloud, we need to change the indices of both point_to_label_dicts, so that they correspond to the
# original point cloud indices
point_to_label_dict_hpr_leftcam_mapped = change_point_indices(point_to_label_dict_hpr_leftcam, hpr_mask_leftcam)
point_to_label_dict_hpr_rightcam_mapped = change_point_indices(point_to_label_dict_hpr_rightcam, hpr_mask_rightcam)

In [ ]:
associations_lr = build_associations(point_to_label_dict_hpr_leftcam_mapped, point_to_label_dict_hpr_rightcam_mapped)
associations_rl = build_associations(point_to_label_dict_hpr_rightcam_mapped, point_to_label_dict_hpr_leftcam_mapped)

Those association can now be visualized on the label predictions. Black areas are areas that are not covered by lidar points, and as such no associations are possible.

In [ ]:
new_label_lr = visualize_associations_in_img(left_label, associations_lr)
new_label_rl = visualize_associations_in_img(right_label, associations_rl)

fig, axes = plt.subplots(2, 2)

images = [left_label, right_label, new_label_rl, new_label_lr]

titles = ["Left Label", "Right Label", "Right Label with Associations", "Left Label with Associations"]

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title(titles[i], fontsize=6)
    ax.set_xticks([]), ax.set_yticks([])

fig.tight_layout(pad=1.0)
fig.set_dpi(300)

plt.show()

Now we would like to apply the associations to the dict that maps point cloud indices to the instance colors.

In [ ]:
point_to_label_dict_hpr_leftcam_associated = apply_associations_to_dict(point_to_label_dict_hpr_leftcam_mapped, associations_lr)
point_to_label_dict_hpr_rightcam_associated = apply_associations_to_dict(point_to_label_dict_hpr_rightcam_mapped, associations_rl)

Now we can merge the point_to_label dict that was associated with the other point_to_label dict to get a merged dict. As of now, this is not sensibly working with HPR, because we might be adding points that were previously removed from HPR. In later steps, this would lead to the program crashing

In [ ]:
merged_labels_lr = merge_label_predictions(point_to_label_dict_hpr_leftcam_associated, point_to_label_dict_hpr_rightcam_mapped, method='iou')
merged_labels_rl = merge_label_predictions(point_to_label_dict_hpr_rightcam_associated, point_to_label_dict_hpr_leftcam_mapped, method='iou')

From this merged dict we can filter the point cloud and color it accordinly!

In [ ]:
pcd_merged_labels_lr = filter_points_from_dict(pcd, merged_labels_lr)
pcd_merged_labels_lr = color_pcd_with_labels(pcd_merged_labels_lr, merged_labels_lr)

pcd_merged_labels_rl = filter_points_from_dict(pcd, merged_labels_rl)
pcd_merged_labels_rl = color_pcd_with_labels(pcd_merged_labels_rl, merged_labels_rl)

Now, we can visualize the merged point cloud with colors

In [ ]:
#o3d.visualization.draw_geometries([pcd_merged_labels_lr])

The next step is to merge multiple point clouds from consecutive timesteps. For this we need to propagate the label maps into the next timeframe. To account for limited compute capabilities, we might need to downsample label maps before building the associations

In [ ]:
scale_factor = 0.25

left_label_t0 = left_label
left_label_t0_downsampled = cv2.resize(left_label, (int(scale_factor * left_label_t0.shape[1]), int(scale_factor * left_label_t0.shape[0])), interpolation = cv2.INTER_NEAREST)

left_label_t1_PIL = dataset.get_sam_label(left_cam, points_index+1)
left_label_t1 = cv2.cvtColor(np.array(left_label_t1_PIL), cv2.COLOR_RGB2BGR)
left_label_t1_downsampled = cv2.resize(left_label_t1, (int(scale_factor * left_label_t1.shape[1]), int(scale_factor * left_label_t1.shape[0])), interpolation = cv2.INTER_NEAREST)

association = build_associations_across_timesteps(left_label_t1_downsampled, left_label_t0_downsampled)
left_label_t1_associated = visualize_associations_in_img(left_label_t1, association)

In [ ]:
fig, axes = plt.subplots(3, 1)

images = [left_label_t0, left_label_t1_associated, left_label_t1]

titles = ["Left Label Timestep=t", "Left Label Timestep=t+1 with associations", "Left Label Timestep=t+1"]

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title(titles[i], fontsize=6)
    ax.set_xticks([]), ax.set_yticks([])

fig.tight_layout(pad=1.0)
fig.set_dpi(100)

plt.show()

This principle can be applied to multiple timesteps in a row, applied on the point clouds and then merged together. This is implemented in the function merged_sequence and is applied below (This takes around sequence_length * 3 seconds on my MacBook M1)

In [ ]:
start_index = 58
sequence_length = 10

label_history, pcd_merge = merged_sequence(dataset, start_index, sequence_length)

In [ ]:
#o3d.io.write_point_cloud("pcd_merge.pcd", pcd_merge, write_ascii=False, compressed=False, print_progress=False)
#o3d.visualization.draw_geometries([pcd_merge])

Here we can visualize the evolution of propagated label maps:

In [ ]:
fig, axes = plt.subplots(10, 1)

for i, ax in enumerate(axes.flat):
    ax.imshow(label_history[i])
    ax.set_xticks([]), ax.set_yticks([])

fig.tight_layout(pad=1.0)
fig.set_dpi(500)

plt.show()